# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [1]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative_plan,
    generate_narrative,
    quality_check,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_person, Person

import copy

# Batch Generation

In [16]:
'''# cost prio, $5.03 (prev $1.24)
test_people_2 = generate_batch_parallel(n=20, model="gpt-5.2", workers=20)'''

Sampling 20 people...
Generating 20 people with 20 parallel workers...


100%|███████████████████████████████████████████| 20/20 [04:13<00:00, 12.69s/it]

Done: 20 generated


In [18]:
'''with open('test_examples_2.pkl', 'wb') as f:
    dill.dump(test_people_2 + olds,f)
%run export.py test_examples_2.pkl'''

In [2]:
with open('test_examples_2.pkl', 'rb') as f:
       examples = dill.load(f)

In [11]:
for i in [0,34,36]:
    print(examples[i].messages)
    print('\n\n\n\n')

[{'role': 'user', 'content': 'This is part of a "random lives" project - simulating randomly selecting a person from human history.\n\nFor this historical person, you will be asked demographic questions. For each question, provide a probability distribution\nrepresenting how common each option was among people matching this person\'s known characteristics\n(birth time, location, age, sex, personality, lifestyle, etc.). Each question should be answered conditional on all previously\ngenerated information.\n\nYOUR GOAL: Estimate the TRUE HISTORICAL FREQUENCIES, not what seems interesting or diverse.\n- If 90% of people in this demographic had characteristic X, assign it 90% probability\n- Focus on ordinary people, not exceptional individuals or elites\n- Boring and repetitive answers are often historically correct\n- Some personality extremes can represent substantial limitations and strongly influence a person\'s life trajectory\n\nTECHNICAL REQUIREMENTS:\n- Probabilities must sum to ex